In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from NetLiquidity import nlq_clean as nq
from NetLiquidity import nlq_chart as plotit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/purpy-furcat/Documents/Code/Bootleg_Macro/MacroBackend /home/purpy-furcat/Documents/Code/Bootleg_Macro


INFO - tedata package initialized successfully!
/home/purpy-furcat/miniconda3/envs/bm/lib/python3.11/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'validate_by_name'
  warnings.warn(message, UserWarning)


System information:  linux , directory delimiter:  / , working directory:  /home/purpy-furcat/Documents/Code/Bootleg_Macro/MacroBackend/Glassnode
System information:  linux linux , directory delimiter:  / , working directory:  /home/purpy-furcat/Documents/Code/Bootleg_Macro/Liquidity/NetLiquidity


/home/purpy-furcat/miniconda3/envs/bm/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/purpy-furcat/Documents/Code/Bootleg_Macro/Liquidity/NetLiquidity/nlq.py:63: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(_norm_cell)


In [2]:
inputs = pd.read_excel(r"/home/purpy-furcat/Documents/Code/Bootleg_Macro/Liquidity/NetLiquidity/NetLiquidity_InputParams.xlsx", index_col=0)
inputs

/home/purpy-furcat/miniconda3/envs/bm/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Additional FRED Data,Display Individually,Comparison Assets,Price API,Comp. Asset Name,TraceColor,Correlation Periods
Index,,,,,,,
1,WRESBAL,yes,"BTCUSD,INDEX",tv,Bitty,orangered,90.0
2,MTSDS133FMS,yes,"SPX,INDEX",tv,S & P 500,black,180.0
3,NaN,NaN,"TLT,NASDAQ",tv,TLT,green,365.0
4,NaN,NaN,NaN,NaN,NaN,NaN,730.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Technicals,NaN,NaN,Information on how to set parameters. More hel...,NaN,NaN,NaN,NaN
API Key,NaN,NaN,Paste your FRED API key in here. Get free at F...,NaN,NaN,NaN,NaN
SaveFREDData,yes,Whether or not to save the series pulled from ...,NaN,NaN,NaN,NaN,NaN
NLQ Source,pull_latest_data,"Not yet in use, leave blank. May be employed i...",NaN,NaN,NaN,NaN,NaN


In [3]:
start_date = inputs.loc["Start date", "Additional FRED Data"]
start_date

'2021-01-01'

In [4]:
nlq_datas = nq.NetLiquidity()

/home/purpy-furcat/miniconda3/envs/bm/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


=== Fetching Core NLQ Data ===
Using total Fed balance sheet (WALCL)
Fetching FRED data from 2021-01-01 to 2026-01-12
Fetching FRED series: WALCL - Fed Total Assets (Weekly)
Looking for data from source:  fred data code:  WALCL
FRED API response:  200
Saved WALCL to /home/purpy-furcat/Documents/Code/Bootleg_Macro/User_Data/FRED_Data/WALCL.xlsx
Successfully fetched WALCL: 262 observations
Fetching FRED series: RRPONTSYD - Reverse Repo Facility (Daily)
Looking for data from source:  fred data code:  RRPONTSYD
FRED API response:  200
Saved RRPONTSYD to /home/purpy-furcat/Documents/Code/Bootleg_Macro/User_Data/FRED_Data/RRPONTSYD.xlsx
Successfully fetched RRPONTSYD: 1253 observations
Fetching FRED series: WTREGEN - Treasury General Account - FRED (Weekly)
Looking for data from source:  fred data code:  WTREGEN
FRED API response:  200
Saved WTREGEN to /home/purpy-furcat/Documents/Code/Bootleg_Macro/User_Data/FRED_Data/WTREGEN.xlsx
Successfully fetched WTREGEN: 262 observations
Fetching TGA 

In [5]:
nq_results = nlq_datas.calculate_all()


=== Calculating All NLQ Versions ===
Created daily index: 2021-01-01 to 2026-01-12 (1838 days)
Calculating NLQ (weekly, raw FRED data)...
NLQ Weekly calculated: 259 observations
Latest value: $5772.87 billion
Calculating NLQ (daily, Treasury TGA)...
NLQ Daily (Treasury) calculated: 1833 observations
Latest value: $5776.27 billion


In [6]:
nq_results.keys()

dict_keys(['nlq_weekly', 'nlq_daily_treasury', 'fed_balance_sheet_daily', 'reverse_repo_daily', 'tga_fred_daily', 'tga_treasury_daily'])

In [7]:
netliq = nq_results["nlq_daily_treasury"]

In [8]:
from MacroBackend import charting_plotly as chp
from MacroBackend import Pull_Data as pul

In [9]:
btc = pul.dataset()
btc.get_data("tv", "BTCUSD", start_date="2014-09-17", exchange_code="INDEX")

you are using nologin method, data you access may be limited


Looking for data from source:  tv data code:  BTCUSD
Number of days of data for  BTCUSD :  4135
Pulling data from trading view. Ticker:  BTCUSD , exchange:  INDEX 
Data time frame:  Daily D , number of days:  4135 , fraction of max data pull length:  0.827 , number of datapoints:  4135 , loop:  0
multi_attempt_pull function: symbol: BTCUSD, exchange: INDEX, collection_method: 0, attempts: 2, timeout: 60
Symbol:Exchange:  INDEX:BTCUSD 
 time interval:  1D 
 number of bars requested:  4135
{'date': 'Mon, 12 Jan 2026 01:53:03 GMT', 'connection': 'upgrade', 'sec-websocket-accept': 'G5qL/1v7jMUkhAxzNn0iOTcJsHg=', 'upgrade': 'websocket', 'server': 'tv'} 101 101
data pull completed for BTCUSD
Data received length:  4135 start date:  2014-09-18 10:00:00 end date:  2026-01-12 11:00:00 4134 days 01:00:00
Data pulled from TV for ticker:  BTCUSD


In [10]:
bitty = btc.data
bitty.plot()

/home/purpy-furcat/miniconda3/envs/bm/lib/python3.11/site-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




In [12]:
figdog = plotit.plot_nlq_dual_axis(nlq_data=netliq, asset_data={"Bitcoin": bitty.loc["01-01-2021":]}, title="Net Liquidity vs Assets")

In [19]:
# Example 2: Chart with custom equally-spaced ticks
import Liquidity.NetLiquidity.nlq_chart as plot


fig2, ax_l2, ax_r2 = plot.plot_nlq_dual_axis(
    nlq_data=netliq,
    asset_data={"Bitcoin": bitty.loc["01-01-2021":]}, 
    title="Net Liquidity vs BTC",
    ma_period=50,
    left_scale='log',
    right_scale='log',
    custom_ticks_left=8,
    custom_ticks_right=10,
    left_tick_prefix='$',
    right_tick_prefix='$',
    left_ticks_int=True,
    right_ticks_int=True
)
fig2.show()

In [21]:
    # Example 3: Using equal_spaced_ticks function for plotly
tick_config = plot.equal_spaced_ticks(
    num_ticks=10,
    data=netliq,
    scale='log',
    lab_prefix='$',
    lab_suffix='B',
    return_format='plotly'
)
print("Plotly tick config:", tick_config)

Plotly tick config: {'tickmode': 'array', 'tickvals': [5550.768, 5704.107899376706, 5861.683811633221, 6023.612756574516, 6190.014986676251, 6361.014076387305, 6536.737013899196, 6717.314295450739, 6902.880022237751, 7093.572000000002], 'ticktext': ['$5550.77B', '$5704.11B', '$5861.68B', '$6023.61B', '$6190.01B', '$6361.01B', '$6536.74B', '$6717.31B', '$6902.88B', '$7093.57B']}


In [30]:
fig = chp.dual_axis_basic_plot(primary_data=netliq, secondary_data=bitty.loc["01-01-2021":], template="ggplot2",
                               log_primary=True, log_secondary=True, title="Net Liquidity vs Bitcoin Price",
                               primary_yaxis_title="Net liquidity (USD Billions)", secondary_yaxis_title=" Price (USD)",     custom_ticks_primary=8,
    custom_ticks_secondary=10, height = 600,
    primary_tick_prefix='$',
    secondary_tick_prefix='$',
    primary_ticks_int=True,
    secondary_ticks_int=True)
fig.show()
